In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from AnsatzLacs import *
import numpy as np
from CCAUtil import *
import PurificationUtil

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2
ref_size = 0 # size of the reference state (for purification)
num_layers = 2
qcbm_layers = 2
N = 2 ** sys_size

In [ ]:
# define hamiltonian 
coefs = [1,1,1]
paulis = ['ZZ','XI','IX']
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in paulis}

In [ ]:
# precompute a couple things
trace_H_2 = N * np.sum(np.square(coefs)) # Tr[H^2]

# Tr[H]
identity_string = 'I' * sys_size
if identity_string in paulis:
    trace_H = N * coefs[paulis.index(identity_string)]
else:
    trace_H = 0

In [ ]:
def hamiltonian(coefs,paulis):
    matrix = np.zeros((N,N), dtype = np.cdouble)
    for i in range(len(coefs)):
        matrix += coefs[i] * get_pauli_prod(paulis[i])
    return matrix

def minimum_eigenvalue(coefs,paulis):
    matrix = hamiltonian(coefs,paulis)
    eigvals = np.linalg.eigvals(matrix)
    return np.amin(np.real(eigvals))

expected = minimum_eigenvalue(coefs,paulis)
print("True Minimum Eigenvalue:", expected)

In [ ]:
def get_exp(pauli_string, ansatz, qcbm_shots, shots):
    """
    Calculate terms like Tr[Hρ] where H is some hamiltonian 
    and ρ is given by paramatrized quantum circuits
    """
    return get_exp_helper(expectation_unitary_dict, pauli_string, ansatz, qcbm_shots, shots)

In [ ]:
# customized SPSA optimizer
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation * delta)
    minus = fun(params - perturbation * delta)
    grad = (plus - minus) / (2 * perturbation * delta)

    norm = np.linalg.norm(grad)
    if norm > 1:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .99, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
def dual_cost_function_full(params):
    # update parameters
    eta = params[0]
    nu = params[1]
    omega_params = params[2:]
    omega_ansatz.set_params(omega_params)
    
    # Objective: eta
    obj = eta
    
    # Penalty: ||H - eta*I - mu*sigma||^2
    tr_H_omega = 0
    for i in range(len(coefs)):
        tr_H_omega += coefs[i] * get_exp(paulis[i], omega_ansatz, qcbm_shots, num_shots)
    
    tr_omega_2 = get_purity(omega_ansatz, qcbm_shots)
        
    penalty = trace_H_2 + eta**2*N + nu**2*tr_omega_2 - 2*eta*trace_H - 2*nu*tr_H_omega + 2*eta*nu
    
    # Put all of it together
    cost = obj - c * penalty
   
    return cost, obj, penalty

def dual_cost_function(params):
    cost, obj, penalty = dual_cost_function_full(params)
    return -cost

In [ ]:
def dual_callback(x, iteration):
    
    global lr
    
    fx, obj, penal = dual_cost_function_full(x)
    
    dual_iterations.append(iteration)
    dual_costs.append(fx)
    dual_penalties.append(penal)
    dual_objs.append(obj)
    
    if (iteration%100==0):
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), "Objective: {:.3g}".format(obj),
          "Penalty: {:.3g}".format(penal), "C: {:.2g}".format(c), "LR: {:.2g}".format(lr), sep = '\t')
        
     # change the learning rate dynamically
    if fx < dual_costs[-2]:
        lr = max(lr / 2, lr_min)
    else:
        lr = min(lr * 1.5, lr_max)

In [ ]:
num_shots = 1e12
qcbm_shots = 1e12

max_iter = 20 * 1000
c = 10
lr_min = 5e-4
lr_max = 1e-1
perturbation = 1e-4

In [ ]:
iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []
record = 0
num_train = 10

for i in range(num_train):
    print(f"\nExpected Value: {round(expected,5)}")
    lr = lr_max
    eta = 0
    nu = 1
    
    np.random.seed(i)
    omega_ansatz = create_cca("general", sys_size, ref_size, num_layers, qcbm_layers)

    init_params = [eta, nu, *omega_ansatz.get_params()]
    init_cost, init_obj, init_penal = dual_cost_function_full(init_params)
    
    dual_iterations = [0]
    dual_costs = [init_cost]
    dual_penalties = [init_penal]
    dual_objs = [init_obj]
    
    print(f"\nIteration: {0}", "Loss: {:.4g}".format(init_cost), "Objective: {:.3g}".format(init_obj),
              "Penalty: {:.3g}".format(init_penal), "C: {:.2g}".format(c), sep = '\t')
    result = gradient_descent(dual_cost_function, init_params, get_gradient, max_iter, callback=dual_callback)
    
    iterations_dual_outer.append(dual_iterations)
    costs_dual_outer.append(dual_costs)
    objs_dual_outer.append(dual_objs)
    penalties_dual_outer.append(dual_penalties)
    
    if abs(dual_costs[-1] - expected) <= 0.05:
        dir_name = f"VQE_CCA_dual_data"
        problem_name = "VQE"
        filename = f"Init_{i}"
        save_data_to_file(dual_costs, dual_penalties, dual_objs, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}") 

# Primal Optimization

In [ ]:
def get_exp_primal(pauli_string, ansatz, shots):
    return PurificationUtil.get_exp_helper(expectation_unitary_dict, pauli_string, ansatz, shots)

In [ ]:
def primal_cost_function(params):
    rho_ansatz.set_params(params)
    
    cost = 0
    for i in range(len(coefs)):
        cost += coefs[i] * get_exp_primal(paulis[i],rho_ansatz, num_shots)
    return cost

In [ ]:
def primal_callback(x, iteration):
    fx = primal_cost_function(x)

    primal_iterations.append(iteration)
    primal_costs.append(fx)
    
    if (iteration%100==0):
        print(f"Iteration: {iteration}", "Loss: {:.4g}".format(fx), sep = '\t')

In [ ]:
num_shots = 1e12
qcbm_shots = 1e12

max_iter = 5 * 1000
c = 10
perturbation = 1e-4

In [ ]:
iterations_primal_outer = []
costs_primal_outer = []
num_train = 10
record = 0

for i in range(num_train):
    lr = 5e-3

    np.random.seed(i)
    rho_ansatz = create_ansatz("general", sys_size, ref_size, num_layers) # we only need pure state
    
    init_params = rho_ansatz.params
    init_cost = primal_cost_function(init_params)
    primal_iterations = [0]
    primal_costs = [init_cost]
    
    print(f"\nExpected Value: {round(expected,5)}")
    print(f"Iteration {i}, Initial cost : {round(init_cost, 4)}")
    result = gradient_descent(primal_cost_function, init_params, get_gradient, max_iter, callback=primal_callback)
    
    iterations_primal_outer.append(primal_iterations)
    costs_primal_outer.append(primal_costs)
    
    if abs(primal_costs[-1] - expected) <= 0.05:
        dir_name = f"VQE_CCA_primal_data"
        problem_name = "VQE"
        filename = f"Init_{i}"
        save_data_to_file(primal_costs, None, None, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}") 
    

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 100
window_size_dual = 100

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(-8, 3)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()